## Preparation of the data and model

In [10]:
dataset_name = "BAShapes"

#### BA-Shapes, 2 layers 1 head

In [11]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "2L1H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 2 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: 0.0933
		spearmanr: 0.0829
		kendalltau: 0.0642
	ΔNE:
		pearsonr: -0.1191
		spearmanr: -0.1067
		kendalltau: -0.0799
	ROC_AUC:
		0.8846
attattribute_sim:
	ΔPC:
		pearsonr: 0.1047
		spearmanr: 0.1101
		kendalltau: 0.0748
	ΔNE:
		pearsonr: -0.1215
		spearmanr: -0.1295
		kendalltau: -0.0919
	ROC_AUC:
		0.8632
avgatt:
	ΔPC:
		pearsonr: 0.0416
		spearmanr: 0.0603
		kendalltau: 0.0391
	ΔNE:
		pearsonr: -0.0543
		spearmanr: -0.0682
		kendalltau: -0.0468
	ROC_AUC:
		0.7092
random:
	ΔPC:
		pearsonr: 0.0125
		spearmanr: 0.0073
		kendalltau: 0.0052
	ΔNE:
		pearsonr: -0.0125
		spearmanr: -0.0074
		kendalltau: -0.0053
	ROC_AUC:
		0.4946


#### BA-Shapes, 3 layers 1 head

In [12]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "3L1H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 3 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: 0.2837
		spearmanr: 0.0340
		kendalltau: 0.0234
	ΔNE:
		pearsonr: -0.2940
		spearmanr: -0.0398
		kendalltau: -0.0247
	ROC_AUC:
		0.9724
attattribute_sim:
	ΔPC:
		pearsonr: 0.3002
		spearmanr: 0.0382
		kendalltau: 0.0259
	ΔNE:
		pearsonr: -0.3026
		spearmanr: -0.0460
		kendalltau: -0.0275
	ROC_AUC:
		0.9594
avgatt:
	ΔPC:
		pearsonr: 0.1688
		spearmanr: 0.0566
		kendalltau: 0.0420
	ΔNE:
		pearsonr: -0.1730
		spearmanr: -0.0692
		kendalltau: -0.0513
	ROC_AUC:
		0.8203
random:
	ΔPC:
		pearsonr: 0.0029
		spearmanr: 0.0039
		kendalltau: 0.0029
	ΔNE:
		pearsonr: -0.0043
		spearmanr: -0.0048
		kendalltau: -0.0035
	ROC_AUC:
		0.5024


#### BA-Shapes, 2 layers 2 head

In [13]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "2L2H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 2 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: 0.0887
		spearmanr: -0.0227
		kendalltau: -0.0155
	ΔNE:
		pearsonr: -0.0900
		spearmanr: 0.0439
		kendalltau: 0.0175
	ROC_AUC:
		0.9205
attattribute_sim:
	ΔPC:
		pearsonr: 0.1249
		spearmanr: 0.0011
		kendalltau: -0.0020
	ΔNE:
		pearsonr: -0.1089
		spearmanr: 0.0128
		kendalltau: 0.0031
	ROC_AUC:
		0.8987
avgatt:
	ΔPC:
		pearsonr: 0.0497
		spearmanr: 0.0200
		kendalltau: 0.0143
	ΔNE:
		pearsonr: -0.0253
		spearmanr: -0.0032
		kendalltau: -0.0020
	ROC_AUC:
		0.7267
random:
	ΔPC:
		pearsonr: -0.0077
		spearmanr: 0.0042
		kendalltau: 0.0030
	ΔNE:
		pearsonr: 0.0024
		spearmanr: -0.0074
		kendalltau: -0.0052
	ROC_AUC:
		0.5035


#### BA-Shapes, 3 layers 2 head

In [14]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "3L2H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 3 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: 0.1566
		spearmanr: -0.0574
		kendalltau: -0.0550
	ΔNE:
		pearsonr: -0.3278
		spearmanr: 0.0768
		kendalltau: 0.0724
	ROC_AUC:
		0.9785
attattribute_sim:
	ΔPC:
		pearsonr: 0.0288
		spearmanr: -0.0614
		kendalltau: -0.0534
	ΔNE:
		pearsonr: -0.1793
		spearmanr: 0.0774
		kendalltau: 0.0684
	ROC_AUC:
		0.9704
avgatt:
	ΔPC:
		pearsonr: 0.0112
		spearmanr: -0.0413
		kendalltau: -0.0313
	ΔNE:
		pearsonr: -0.0849
		spearmanr: 0.0504
		kendalltau: 0.0382
	ROC_AUC:
		0.9095
random:
	ΔPC:
		pearsonr: 0.0119
		spearmanr: 0.0089
		kendalltau: 0.0070
	ΔNE:
		pearsonr: -0.0129
		spearmanr: -0.0086
		kendalltau: -0.0067
	ROC_AUC:
		0.5040


### BA-Shapes, 2 layers 4 head

In [15]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "2L4H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 2 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: -0.0478
		spearmanr: -0.0334
		kendalltau: -0.0244
	ΔNE:
		pearsonr: -0.0031
		spearmanr: -0.0831
		kendalltau: -0.0717
	ROC_AUC:
		0.9333
attattribute_sim:
	ΔPC:
		pearsonr: -0.0244
		spearmanr: 0.0033
		kendalltau: -0.0063
	ΔNE:
		pearsonr: -0.0301
		spearmanr: -0.1201
		kendalltau: -0.0919
	ROC_AUC:
		0.8992
avgatt:
	ΔPC:
		pearsonr: 0.0093
		spearmanr: -0.0086
		kendalltau: -0.0021
	ΔNE:
		pearsonr: -0.0165
		spearmanr: 0.0012
		kendalltau: -0.0054
	ROC_AUC:
		0.7060
random:
	ΔPC:
		pearsonr: 0.0010
		spearmanr: 0.0115
		kendalltau: 0.0081
	ΔNE:
		pearsonr: 0.0044
		spearmanr: -0.0057
		kendalltau: -0.0040
	ROC_AUC:
		0.5043


### BA-Shapes, 3 layers 4 head

In [16]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "3L4H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 3 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: 0.2041
		spearmanr: 0.0016
		kendalltau: 0.0000
	ΔNE:
		pearsonr: -0.3956
		spearmanr: -0.1713
		kendalltau: -0.1644
	ROC_AUC:
		0.9736
attattribute_sim:
	ΔPC:
		pearsonr: 0.0499
		spearmanr: -0.0027
		kendalltau: -0.0043
	ΔNE:
		pearsonr: -0.2494
		spearmanr: -0.1213
		kendalltau: -0.1202
	ROC_AUC:
		0.9615
avgatt:
	ΔPC:
		pearsonr: 0.0200
		spearmanr: -0.0020
		kendalltau: -0.0011
	ΔNE:
		pearsonr: -0.1180
		spearmanr: 0.0561
		kendalltau: 0.0479
	ROC_AUC:
		0.8383
random:
	ΔPC:
		pearsonr: -0.0035
		spearmanr: -0.0049
		kendalltau: -0.0038
	ΔNE:
		pearsonr: 0.0033
		spearmanr: 0.0042
		kendalltau: 0.0032
	ROC_AUC:
		0.5022


### BA-Shapes, 2 layers 8 head

In [17]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "2L8H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 2 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: 0.3202
		spearmanr: 0.2724
		kendalltau: 0.1980
	ΔNE:
		pearsonr: -0.4083
		spearmanr: -0.2829
		kendalltau: -0.2041
	ROC_AUC:
		0.9126
attattribute_sim:
	ΔPC:
		pearsonr: 0.3342
		spearmanr: 0.2518
		kendalltau: 0.1774
	ΔNE:
		pearsonr: -0.4156
		spearmanr: -0.2600
		kendalltau: -0.1875
	ROC_AUC:
		0.8704
avgatt:
	ΔPC:
		pearsonr: 0.0760
		spearmanr: 0.1060
		kendalltau: 0.0755
	ΔNE:
		pearsonr: -0.1213
		spearmanr: -0.0997
		kendalltau: -0.0710
	ROC_AUC:
		0.6727
random:
	ΔPC:
		pearsonr: 0.0241
		spearmanr: 0.0169
		kendalltau: 0.0121
	ΔNE:
		pearsonr: -0.0208
		spearmanr: -0.0190
		kendalltau: -0.0137
	ROC_AUC:
		0.5170


### BA-Shapes, 3 layers 8 head

In [18]:
# Import the utility functions
import torch
from src.experiment_utils import FaithfulnessExperiment, FaithfulnessExperimentAnalysis

device = torch.device("cpu")
dataset_folder = "/workspace/Datasets"
model_folder = "/workspace/Models"
config = "3L8H"

# Load the network data locally (for reproduction)
data = torch.load(f"{dataset_folder}/{dataset_name}.pt").to(device)
# Load model as a whole
model = torch.load(f"{model_folder}/GAT_{dataset_name}_{config}.pt").to(device)
model.eval()

with torch.no_grad():
    _  = model(data.x, data.edge_index, return_att = True)
    att = list(model.att)
    

# Define the experiment
faithfulness_experiment = FaithfulnessExperiment(
    model = model,
    data = data,
    device = device,
    )

faithfulness_experiment.set_target_nodes(
    list(range(300, 700))
)
attribute_dict = faithfulness_experiment.get_attributions()
intervention_dict = faithfulness_experiment.model_intervention()

analysis = FaithfulnessExperimentAnalysis(
    attribution_dict = attribute_dict,
    intervention_dict = intervention_dict,
)
analysis.generate_random_baseline()
result = analysis.get_full_analysis()
analysis.print_result(result=result)

# Save the result
experiment_artifacts_folder = "/workspace/Experimental_Artifacts"
torch.save(attribute_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Attributions.pt")
torch.save(intervention_dict, f"{experiment_artifacts_folder}/Faithfulness_GAT_{dataset_name}_{config}_Interventions.pt")
experiments_folder = "/workspace/Experimental_Results"
torch.save(result, f"{experiments_folder}/Faithfulness_GAT_infection_{config}.pt")

Target nodes set...
Generating edge lists within 3 hops of target nodes...
...Done
Getting attributions for 400 nodes...
...Done
Getting model intervention for 400 nodes...
attattribute:
	ΔPC:
		pearsonr: 0.3743
		spearmanr: 0.1756
		kendalltau: 0.1590
	ΔNE:
		pearsonr: -0.4306
		spearmanr: -0.2042
		kendalltau: -0.1924
	ROC_AUC:
		0.9630
attattribute_sim:
	ΔPC:
		pearsonr: 0.3710
		spearmanr: 0.1773
		kendalltau: 0.1533
	ΔNE:
		pearsonr: -0.4390
		spearmanr: -0.1982
		kendalltau: -0.1788
	ROC_AUC:
		0.9581
avgatt:
	ΔPC:
		pearsonr: 0.1674
		spearmanr: 0.0851
		kendalltau: 0.0632
	ΔNE:
		pearsonr: -0.2071
		spearmanr: -0.0508
		kendalltau: -0.0346
	ROC_AUC:
		0.8490
random:
	ΔPC:
		pearsonr: 0.0033
		spearmanr: -0.0046
		kendalltau: -0.0035
	ΔNE:
		pearsonr: -0.0046
		spearmanr: 0.0019
		kendalltau: 0.0015
	ROC_AUC:
		0.4991
